<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

     |████████████████████████████████| 727kB 7.7MB/s 
     |████████████████████████████████| 194kB 16.1MB/s 
     |████████████████████████████████| 1.2MB 22.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 12.8MB 241kB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/testing'),Path('/root/.fastai/data/mnist_png/training')]

In [5]:
Path.BASE_PATH = path

In [6]:
(path/'training').ls()

(#10) [Path('training/5'),Path('training/2'),Path('training/3'),Path('training/4'),Path('training/1'),Path('training/6'),Path('training/8'),Path('training/0'),Path('training/7'),Path('training/9')]

In [7]:
categories = [x for x in range(0,10)]
categories

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
train_x = torch.tensor([])
train_y = torch.tensor([])
valid_x = torch.tensor([])
valid_y = torch.tensor([])

In [9]:
for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [10]:
for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [11]:
train_x = train_x.view(-1,28*28).float()/255
valid_x = valid_x.view(-1,28*28).float()/255
train_y = train_y.unsqueeze(1).int()
valid_y = valid_y.unsqueeze(1).int()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [12]:
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [13]:
simple_net = nn.Sequential(
    nn.Linear(28*28,50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,1),
)

In [14]:
pred = simple_net(train_x[1])
pred_class = (pred == torch.max(pred)).nonzero(as_tuple=True)[0]
pred_prob = pred[pred_class]
pred_class, pred_prob

(tensor([0]), tensor([-0.0047], grad_fn=<IndexBackward>))

In [15]:
def loss_f(pred, label):
  pred = pred.sigmoid()
  pred_prob = pred[label.item()]
  return torch.where(label == 0, 1-pred, pred).mean()

In [16]:
class optimizer():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def calc_grad(self, pred,label):
    loss = loss_f(pred,label)
    loss.backward()
  def step(self):
    for p in self.parameters:
      p.data -= p.grad
  def grads_zero(self):
    for p in self.parameters:
      p.grad = None


In [17]:
opt = optimizer(simple_net.parameters(), 0.2)

In [18]:
opt.calc_grad(train_x[0],train_y[0])
opt.grads_zero()

RuntimeError: ignored

In [ ]:
for dx, dy in dls:
  loss = loss_f(dx,dy)
  opt.calc_grad(loss)
  opt.step()
  opt.grads_zero()